# Notebook layout

Note the user can activate 'Run All', however, most parts of the code have been commented out due to the requirement of all the supplied data (including coordinates) for the particular code - which is confidential in nature and cannot be uploaded to a shared drive. The processed data, that retains confidentiality, has been loaded into the code at various stages to allow notebook functionality.

The notebook contains the following sections that refer to the Modelling report layout for Group 13:
- 1.3 Feature engineering
    - Step 1) Classify active weather stations
    - Step 2) Match active stations to pests
    - Step 3) Calculate average readings per month
    - Step 4) Determine actual readings per month
    - Step 5) Calculate difference between actual and average readings per month
    - Step 6) Format data for modelling

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
from functions import *

# Feature Engineering

## Preparation

Note that the code is run for four parameters: 
- rainfall, 
- temperature_max, 
- temperature_min, and 
- temperature_diff.

Each time the variable of note is uncommented from the cell below, with all other variables of note commented out. 

In [2]:
variableOfNote = 'rainfall'
# variableOfNote = 'temperature_max'
# variableOfNote = 'temperature_min'
# variableOfNote = 'temperature_diff'

exportFolder = 'C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/Modeling/' + variableOfNote + '/'

## Step 1) Classify active weather stations

### 1.1) Load weather data

In [3]:
# if variableOfNote != 'temperature_diff':
#     variableDf = pd.read_csv('C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/RawData/weather_1950_2019_v3.csv', usecols=["clim_no", variableOfNote, "record_date"])    
# else:
#     variableDf = pd.read_csv('C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/RawData/weather_1950_2019_v3.csv', usecols=["clim_no", 'temperature_max', 'temperature_min', "record_date"])

# variableDf["record_date"] = pd.to_datetime(variableDf["record_date"])

# if variableOfNote == 'rainfall':
#     rainAdditional = pd.read_csv('C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/RawData/rain_temp2017_2018.csv', usecols=["clim_no", variableOfNote, "record_date"])
#     rainAdditional["record_date"] = pd.to_datetime(rainAdditional["record_date"], format='%d/%m/%Y')
#     variableDf = variableDf.append(rainAdditional)

#     variableDf[variableOfNote] = pd.to_numeric(variableDf[variableOfNote])
#     variableDf = variableDf[variableDf[variableOfNote] >= 0]

# else:
#     tempAdditional2017 = pd.read_csv('C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/RawData/rain_temp2017_2018.csv', usecols=["clim_no", "Tmax", "Tmin","record_date"])
#     tempAdditional2017["record_date"] = pd.to_datetime(tempAdditional2017["record_date"], format='%d/%m/%Y')
#     tempAdditional2015 = pd.read_csv('C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/Modeling/Cumulative2015temp_R01.csv', usecols=["clim_no", "tmax","tmin" ,"record_date"])
#     tempAdditional2015["record_date"] = pd.to_datetime(tempAdditional2015["record_date"], format='%Y-%m-%d')
#     tempAdditional2017 = tempAdditional2017.rename(columns = {"Tmax":'temperature_max'}) 
#     tempAdditional2015 = tempAdditional2015.rename(columns = {"tmax":'temperature_max'})
#     tempAdditional2017 = tempAdditional2017.rename(columns = {"Tmin":'temperature_min'}) 
#     tempAdditional2015 = tempAdditional2015.rename(columns = {"tmin":'temperature_min'})
    
#     if variableOfNote == 'temperature_max':        
#         tempAdditional2017 = tempAdditional2017.drop(columns=["temperature_min"])
#         tempAdditional2015 = tempAdditional2015.drop(columns=["temperature_min"])
#     elif variableOfNote == 'temperature_min':        
#         tempAdditional2017 = tempAdditional2017.drop(columns=["temperature_max"])
#         tempAdditional2015 = tempAdditional2015.drop(columns=["temperature_max"])
    

#     variableDf = variableDf.append(tempAdditional2017)
#     variableDf = variableDf.append(tempAdditional2015) 

#     if variableOfNote == 'temperature_diff':
#         variableDf['temperature_max'] = pd.to_numeric(variableDf['temperature_max'])
#         variableDf = variableDf[variableDf['temperature_max'] != -99.9]

#         variableDf['temperature_min'] = pd.to_numeric(variableDf['temperature_min'])
#         variableDf = variableDf[variableDf['temperature_min'] != -99.9]

#         variableDf['temperature_diff'] = variableDf['temperature_max'] - variableDf['temperature_min']
#         variableDf = variableDf.drop(columns=["temperature_max","temperature_min"])
#     else:
#         variableDf[variableOfNote] = pd.to_numeric(variableDf[variableOfNote])
#         variableDf = variableDf[variableDf[variableOfNote] != -99.9]


# variableDf = variableDf.drop_duplicates()
# variableDf = variableDf.reset_index()
# variableDf['year'] = pd.DatetimeIndex(variableDf['record_date']).year
# variableDf['month'] = pd.DatetimeIndex(variableDf['record_date']).month

# variableDf.head()

variableDfexample = read_from_drive('https://drive.google.com/file/d/1fj5oNiVVF55rhM6NSHMNeI44o4ZgJIfI/view?usp=sharing')
print(variableDfexample)


   Unnamed: 0  index    clim_no  rainfall  record_date  year  month
0           0      0  0303391_W         0        32134  1987     12
1           1      1  0303391_W         0        32135  1987     12
2           2      2  0303391_W         0        32136  1987     12
3           3      3  0303391_W         0        32137  1987     12
4           4      4  0303391_W         0        32138  1987     12


### 1.2) Get number of readings per month per station

Ensure that only stations with complete data for the month are considered in the analysis.

In [4]:
# readingPerMonthCount = variableDf.groupby(["year", "month","clim_no"]).size().reset_index(name="count")

# # Check for completeness in the data
# readingPerMonthCount = (
#     #Feb
#     readingPerMonthCount[((readingPerMonthCount['month'] == 2) & ((readingPerMonthCount['count'] == 28)|(readingPerMonthCount['count'] == 29))) |
#     #30 day months
#     (((readingPerMonthCount['month'] == 4) | (readingPerMonthCount['month'] == 6) | (readingPerMonthCount['month'] == 9) |(readingPerMonthCount['month'] == 11)) & (readingPerMonthCount['count'] == 30)) |    
#     #31 day months
#     (((readingPerMonthCount['month'] == 1) | (readingPerMonthCount['month'] == 3) | (readingPerMonthCount['month'] == 5) |(readingPerMonthCount['month'] == 7)|(readingPerMonthCount['month'] == 8)|(readingPerMonthCount['month'] == 10)|(readingPerMonthCount['month'] == 12)) & (readingPerMonthCount['count'] == 31)) ])

# print(readingPerMonthCount.head())
# readingPerMonthCount.to_csv(exportFolder + 'readingPerMonthCount.csv')

readingPerMonthCountexample = read_from_drive('https://drive.google.com/file/d/1Mfn7D1AFTO-nrDALK--a1ZcMfFMj8cuC/view?usp=sharing')
print('Count of reading per month:')
print(readingPerMonthCountexample)

Count of reading per month:
   Unnamed: 0  year  month    clim_no  count
0           0  1950      1  0001517_A     31
1           1  1950      1  0001517_W     31
2           2  1950      1  0001605_W     31
3           3  1950      1  0001726_W     31
4           4  1950      1  0001813_W     31


### 1.3) Aggregate reading per month per station name

In [5]:
# ReadingPerMonth = variableDf[["year","month","clim_no",variableOfNote]]

# if variableOfNote == 'rainfall':
#     ReadingPerMonth = ReadingPerMonth.groupby(['clim_no','year','month']).sum().reset_index()
# else:
#     ReadingPerMonth = ReadingPerMonth.groupby(['clim_no','year','month']).mean().reset_index()

# print("Reading per month:")
# ReadingPerMonth.head()

ReadingPerMonthexample = read_from_drive('https://drive.google.com/file/d/1XN7cjORpuE3j1gqDHsAnb50yluL60reN/view?usp=sharing')
print("Aggregate reading per month:")
print(ReadingPerMonthexample)

Aggregate reading per month:
   Unnamed: 0    clim_no  year  month  rainfalll
0           0  0001517_8  2000      1       15.8
1           1  0001517_8  2000      2        2.4
2           2  0001517_8  2000      3       54.3
3           3  0001517_8  2000      4       17.9
4           4  0001517_8  2000      5       49.4


#### 1.4) Get reading per month per station Id

In [6]:
# weatherStationIds = pd.read_csv("C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/Modeling/WeatherStationsClean_R03_inMeters.csv", usecols =['Id','clim_no_2']).set_index('clim_no_2')

# readingPerMonthMatched = ReadingPerMonth.set_index('clim_no').join(weatherStationIds)
# readingPerMonthMatched = readingPerMonthMatched.dropna(subset = ['Id'])
# readingPerMonthMatched["Id"] = pd.to_numeric(readingPerMonthMatched["Id"],downcast='integer')

# readingPerMonthMatched = readingPerMonthMatched.set_index('Id')
# print("Reading per month matched to station Id:")
# print(readingPerMonthMatched.head())
# readingPerMonthMatched.to_csv(exportFolder + 'readingPerMonthMatched.csv')

readingPerMonthMatchedexample = read_from_drive('https://drive.google.com/file/d/1Qxc8t0mQWxlxqDpuTduZbPXZ-_w63TGn/view?usp=sharing')
print("Reading per month matched to station Id:")
print(readingPerMonthMatchedexample)

Reading per month matched to station Id:
     Id  year  month  rainfall
0  6140  2000      1      15.8
1  6140  2000      2       2.4
2  6140  2000      3      54.3
3  6140  2000      4      17.9
4  6140  2000      5      49.4


## Step 2) Match active stations to pests

Note that this is done on a month-to-month basis for every year the data is available, using only stations with complete data for that month. Pest inspection are assigned to the closest active weather station for the year-month period.

### 2.1) Distance function

In [7]:
# def getStations(pestData, colName): 
#     readingPerMonthCount = pd.read_csv(exportFolder + 'readingPerMonthCount.csv')
#     weatherStationsAll = pd.read_csv("C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/Modeling/WeatherStationsClean_R03_inMeters.csv", usecols =["Id","clim_no_2","x_m","y_m"]).set_index('clim_no_2')

#     pestLoc = np.array(pestData[["x_m","y_m"]])
#     pestStations = pestData.copy(deep=True)
#     pestStations = pestStations.set_index('Id')

#     for tyear in range(1950,2020):
#         for tmonth in range(1,13):
#             try:
#                 activeStations = readingPerMonthCount[(readingPerMonthCount["year"] == tyear) & (readingPerMonthCount["month"] == tmonth)]                
#                 activeStations = activeStations.set_index('clim_no').join(weatherStationsAll)  
#                 activeStations = activeStations.dropna(subset = ["Id"])   
#                 activeStationLoc = np.array(activeStations[["x_m","y_m"]])

#                 distance = distance_matrix(pestLoc,activeStationLoc, p=2)        
#                 minValues = []

#                 for i in range(0,len(distance)):   
#                     closestStationIndex = np.where(distance[i] == np.amin(distance[i]))[0][0]
#                     minValues.append((pestData.iloc[i,0],int(activeStations.iloc[closestStationIndex,4])))

#                 minValues = pd.DataFrame(minValues, columns = [colName,"StationsId"])
#                 minValues = minValues.rename(columns = {"StationsId":str(tyear)+'_'+str(tmonth)})

#                 pestStations = pestStations.join(minValues.set_index(colName))
#             except:
#                 pass            
#     return  pestStations

### 2.2) Applied to pests

#### 2.2.1) Leptocybe stations

Calculate (note this will take about 1 minute to run complete code):

In [8]:
# # Load files
# leptoAll = pd.read_csv('C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/Modeling/Leptocybe_R01.csv', usecols = ["Id","x_m","y_m"])

# # Run function
# leptoStations = getStations(leptoAll, 'LeptoId') 

# # Export
# leptoStations.to_csv(exportFolder + 'leptoStations.csv')
# leptoStations.head()  

leptoStationsexample = read_from_drive('https://drive.google.com/file/d/1TOT3vil871lRDRCxHTNBAQk7UU0oeR0O/view?usp=sharing').set_index('Id')
print("Lepto Stations:")
print(leptoStationsexample)

Lepto Stations:
            x_m      y_m  1950_1  1950_2  1950_3  1950_4  1950_5  1950_6  \
Id                                                                         
1   3161653.426 -3709227    1067    1067    1067    1067    1067    1067   
2   3351486.694 -2741825    5178    5178    5178    5178    5178    5178   
3   3346585.920 -2721167    5165    5165    5165    5165    5165    5165   
4   3353945.612 -2721145    5173    5173    5173    5173    5173    5173   
5   3361798.798 -2724855    5190    5190    5190    5190    5190    5190   

    1950_7  1950_8  ...  2018_10  2018_11  2018_12  2019_1  2019_2  2019_3  \
Id                  ...                                                      
1     1067    1067  ...     5635     5635     5635    5635    5635    5635   
2     5178    5178  ...     5858     5858     6383    6383    5858    5858   
3     5165    5165  ...     5802     5802     5802    5802    5802    5802   
4     5173    5173  ...     5802     5802     5802    5802   

#### 2.2.2) Sirex stations

Calculate (note this will take about 6 minutes to run complete code):

In [9]:
# # Load files
# sirexAll = pd.read_csv('C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/Modeling/Sirex_R01.csv', usecols = ["Id","x_m","y_m"])

# # Run function
# sirexStations = getStations(sirexAll, 'SirexId') 

# # Export
# sirexStations.to_csv(exportFolder + 'sirexStations.csv')
# sirexStations.head() 

sirexStationsexample = read_from_drive('https://drive.google.com/file/d/1LdWRkvB0I3ekI3uJO-ORagGPGFj-bML8/view?usp=sharing').set_index('Id')
print("Sirex Stations:")
print(sirexStationsexample)

Sirex Stations:
             x_m          y_m  1950_1  1950_2  1950_3  1950_4  1950_5  1950_6  \
Id                                                                              
392  3370640.635 -2618490.615    5360    5360    5360    5360    5360    5360   
391  3371099.272 -2621810.237    5360    5360    5360    5360    5360    5360   
390  3375183.584 -2622777.126    5360    5360    5360    5360    5360    5360   
389  3324792.590 -2631759.843    5334    5334    5334    5334    5334    5334   
388  3357687.499 -2632545.914    5284    5284    5284    5284    5284    5284   

     1950_7  1950_8  ...  2018_10  2018_11  2018_12  2019_1  2019_2  2019_3  \
Id                   ...                                                      
392    5360    5360  ...     6394     6394     6394    6394    6394    6394   
391    5360    5360  ...     6394     6394     6394    6394    6394    6394   
390    5360    5360  ...     6394     6394     6394    6394    6394    6394   
389    5334    5334  

## Step 3) Calculate average readings per month

### 3.1) Monthly average function

In [10]:
# def getAvgPerMonth(pestData, pestStations):
#     readingPerMonthMatched = pd.read_csv(exportFolder + 'readingPerMonthMatched.csv')
    
#     pestReadingPerMonth = pestData.copy(deep=True)
#     pestreadingPerMonthCount = pestData.copy(deep=True)
#     for month in range(1,13):
#         pestReadingPerMonth[[str(month)]] = 0
#         pestreadingPerMonthCount[[str(month)]] = 0

#     pestReadingPerMonth = pestReadingPerMonth.set_index("Id")
#     pestreadingPerMonthCount = pestreadingPerMonthCount.set_index("Id")
   
#     for tyear in range(1950,2020):
#         for tmonth in range(1,13):
#             try:    
#                 tPest = pestStations[['Id',str(tyear)+'_'+str(tmonth)]].reset_index()#Get active stations per inspection
#                 tPest = tPest.drop(columns=['index'])
#                 tPest = tPest.rename(columns = {str(tyear)+'_'+str(tmonth):'stationId'})                

#                 tReading = readingPerMonthMatched[(readingPerMonthMatched["year"] == tyear) & (readingPerMonthMatched["month"] == tmonth)] 
#                 tReading = tReading.rename(columns = {'Id':'stationId'})

#                 tPesttReading = pd.merge(tPest,tReading, on = ['stationId'])
#                 tPesttReading = tPesttReading.set_index('Id')                         

#                 pestReadingPerMonth = pestReadingPerMonth.join(tPesttReading) #join to end of df
#                 pestreadingPerMonthCount = pestreadingPerMonthCount.join(tPesttReading)

#                 pestReadingPerMonth[variableOfNote] = pestReadingPerMonth[variableOfNote].fillna(0)
#                 pestreadingPerMonthCount[variableOfNote] = pestreadingPerMonthCount[variableOfNote].fillna(0)

#                 pestReadingPerMonth[str(tmonth)] = pestReadingPerMonth[str(tmonth)] + pestReadingPerMonth[variableOfNote]   
#                 pestreadingPerMonthCount[str(tmonth)] = pestreadingPerMonthCount[str(tmonth)] + (pestreadingPerMonthCount[variableOfNote] /pestreadingPerMonthCount[variableOfNote] ).fillna(0)

#                 pestReadingPerMonth = pestReadingPerMonth.drop(columns=[variableOfNote,'stationId','month','year'])  
#                 pestreadingPerMonthCount = pestreadingPerMonthCount.drop(columns=[variableOfNote,'stationId','month','year'])
#             except:
#                 pass    
#     for tmonth in range(1,13):
#         pestReadingPerMonth[str(tmonth)] = pestReadingPerMonth[str(tmonth)] / pestreadingPerMonthCount[str(tmonth)]

#     return pestReadingPerMonth

### 3.2) Applied to pests

#### 3.2.1) Leptocybe

In [11]:
# #Load files
# leptoAll = pd.read_csv ('C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/Modeling/Leptocybe_R01.csv', usecols = ["Id","x_m","y_m"])
# leptoStations = pd.read_csv(exportFolder + 'leptoStations.csv')

# #Run function
# leptoReadingPerMonth = getAvgPerMonth(leptoAll, leptoStations)

# #Export
# leptoReadingPerMonth.to_csv(exportFolder + 'leptoReadingPerMonthAvg.csv')
# print("Avg reading per month expected per Leptocybe inspection:")
# print(leptoReadingPerMonth)

leptoReadingPerMonthexample = read_from_drive('https://drive.google.com/file/d/1jUF2R0APMJu714TOLK6VHnLH3_vtF4fn/view?usp=sharing').set_index('Id')
print("Avg reading per month expected per Leptocybe inspection:")
print(leptoReadingPerMonthexample)

Avg reading per month expected per Leptocybe inspection:
            x_m      y_m           1           2           3          4  \
Id                                                                        
1   3161653.426 -3709227  184.791304  158.107353  155.240298  70.325000   
2   3351486.694 -2741825  281.915942  253.226087  176.705797  94.202899   
3   3346585.920 -2721167  253.366667  253.627536  160.823188  80.089552   
4   3353945.612 -2721145  232.666667  212.785507  140.502899  71.626866   
5   3361798.798 -2724855  203.566667  204.975000  120.495652  63.114493   

            5          6          7          8          9          10  \
Id                                                                      
1   32.157812  21.518966  23.684746  36.603030  69.536232  115.970588   
2   31.244615  18.326984  20.763077  19.884375  40.705970   85.459420   
3   28.629508  17.891071  19.112281  16.019048  36.672727   76.092754   
4   27.481356  18.083636  15.652542  12.523810  29.6

#### 3.2.2) Sirex

In [12]:
# #Load files
# sirexAll = pd.read_csv('C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/Modeling/Sirex_R01.csv', usecols = ["Id","x_m","y_m"])
# sirexStations = pd.read_csv(exportFolder + 'sirexStations.csv')

# #Run function
# sirexReadingPerMonth = getAvgPerMonth(sirexAll, sirexStations)

# #Export
# sirexReadingPerMonth.to_csv(exportFolder + 'sirexReadingPerMonthAvg.csv')
# print("Avg reading per month expected per Sirex inspection:")
# print(sirexReadingPerMonth)

sirexReadingPerMonthexample = read_from_drive('https://drive.google.com/file/d/13MkStuk4E3sMqeBUnhbvK_b0998nJZBF/view?usp=sharing').set_index('Id')
print("Avg reading per month expected per Sirex inspection:")
print(sirexReadingPerMonthexample)

Avg reading per month expected per Sirex inspection:
             x_m          y_m           1           2           3          4  \
Id                                                                             
392  3370640.635 -2618490.615  253.917647  237.117647  137.936232  71.481538   
391  3371099.272 -2621810.237  253.917647  252.433823  143.478261  73.232308   
390  3375183.584 -2622777.126  257.823188  265.194203  158.786957  75.230882   
389  3324792.590 -2631759.843  128.982609  121.708696   78.391304  36.492424   
388  3357687.499 -2632545.914  242.079412  254.040580  149.474627  68.797015   

             5          6          7          8          9         10  \
Id                                                                      
392  35.762712  26.805769  23.889091  23.608197  38.664615  66.035294   
391  36.244068  28.119231  24.054545  23.608197  38.664615  66.035294   
390  35.173437  25.859016  23.824138  22.901563  38.398485  71.127536   
389  20.451923  18.70

## Step 4) Determine actual readings per month

### 4.1) Actual readings function

In [13]:
# def getActualReadings(pestData, pestStations, startYear):
#     readingPerMonthMatched = pd.read_csv(exportFolder + 'readingPerMonthMatched.csv')
#     pestIns = pestData[pestData['Year']<2019] #Limit due to available weather data

#     pestInsActuals = pestIns.copy(deep=True)
#     for month in range(1,49):
#         pestInsActuals[[str(month)]] = 0
#     pestInsActuals = pestInsActuals.set_index('Id')
#     pestInsActuals['Year'] = pestInsActuals['Year'].astype('int')

#     for tyear in range(startYear,2019):    
#         yearMax = tyear
#         yearMin = tyear - 3
#         colMonth = 0   

#         pestInsActualsForYear = pestInsActuals[['Year']]
#         pestInsActualsForYear = pestInsActualsForYear[pestInsActualsForYear['Year'] == tyear]        

#         for year in range(yearMin,yearMax + 1):               
#             for month in range(1,13):
#                 colMonth += 1                          
#                 try:                          
#                     #Get station for the month
#                     pestStationsForMonth = pestStations[['Id',str(year)+'_'+str(month)]]
#                     pestStationsForMonth = pestStationsForMonth.rename(columns = {str(year)+'_'+str(month):'stationId'})

#                     #Get rainfall for station for the month
#                     ActualReading = readingPerMonthMatched[(readingPerMonthMatched['year'] == year) & (readingPerMonthMatched['month'] == month)]
#                     ActualReading = ActualReading.rename(columns = {'Id':'stationId'}) 

#                     ActualReading = pd.merge(pestStationsForMonth,ActualReading, on = ['stationId'])
#                     ActualReading['Id'] = ActualReading['Id'].astype('int')
#                     ActualReading = ActualReading.set_index('Id')

#                 except:                     
#                     try:                         
#                         pestStationsForMonth = pestStations[['Id',str(year-1)+'_'+str(month)]]                        
#                         pestStationsForMonth = pestStationsForMonth.rename(columns = {str(year-1)+'_'+str(month):'stationId'}) 

#                         ActualReading = readingPerMonthMatched[(readingPerMonthMatched['year'] == year-1) & (readingPerMonthMatched['month'] == month)]
#                         ActualReading = ActualReading.rename(columns = {'Id':'stationId'}) 
#                         ActualReading = pd.merge(ActualReading,pestStationsForMonth, on = ['stationId'])                        
                                           
#                         ActualReading['Id'] = ActualReading['Id'].astype('int')
#                         ActualReading = ActualReading.set_index('Id')
#                         ActualReading[variableOfNote] = 0                       

#                         avgRange = [-2,-1,1,2]                         

#                         for i in avgRange:
#                             pestStationsForMonthTemp = pestStations[['Id',str(year+i)+'_'+str(month)]]                                                 
#                             pestStationsForMonthTemp = pestStationsForMonthTemp.rename(columns = {str(year+i)+'_'+str(month):'stationId'})
                            

#                             ActualReadingTemp = readingPerMonthMatched[(readingPerMonthMatched['year'] == year+i) & (readingPerMonthMatched['month'] == month)]
#                             ActualReadingTemp = ActualReadingTemp.rename(columns = {'Id':'stationId'}) 
                        
#                             ActualReadingTemp = pd.merge(ActualReadingTemp,pestStationsForMonthTemp, on = ['stationId']) 
                  
#                             # ActualReadingTemp['Id'] = ActualReadingTemp['Id'].astype('int')
#                             ActualReadingTemp = ActualReadingTemp.set_index('Id')                          
#                             ActualReading[variableOfNote] = ActualReading[variableOfNote] + ActualReadingTemp[variableOfNote]
                        
#                         ActualReading[variableOfNote] = ActualReading[variableOfNote]/len(avgRange)
#                         print("No data for", year, month, 'average used.') 
                         
#                     except:
#                         print("No data for", year, month) 
                
#                 pestInsActualsForYearWithReading = pestInsActualsForYear.join(ActualReading)   
#                 pestInsActualsForYearWithReading = pestInsActualsForYearWithReading.drop(columns=['Year','year','month', 'stationId'])

#                 pestInsActuals = pestInsActuals.join(pestInsActualsForYearWithReading)
#                 pestInsActuals[variableOfNote] = pestInsActuals[variableOfNote].fillna(0)                    

#                 pestInsActuals[str(colMonth)] = pestInsActuals[str(colMonth)] + pestInsActuals[variableOfNote]
#                 pestInsActuals = pestInsActuals.drop(columns=[variableOfNote])

#     return pestInsActuals

### 4.2) Applied to pests

#### 4.2.1) Leptocybe

In [14]:
# #Load files
# leptoAll = pd.read_csv('C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/Modeling/Leptocybe_R01.csv',usecols =["Id","Year","Lepto_Y_N"])
# leptoStations = pd.read_csv(exportFolder + 'leptoStations.csv')

# #Run function
# leptoInsActuals = getActualReadings(leptoAll, leptoStations, 2016)

# #Export
# print('Lepto actuals per inspection 3 years prior and including year of inspection:')
# print(leptoInsActuals)
# leptoInsActuals.to_csv(exportFolder + 'leptoInsActuals.csv')

leptoInsActualsExample = read_from_drive('https://drive.google.com/file/d/1jLR7OMpkdCIN3Z_8eO698rJ2SUrOIYjg/view?usp=sharing').set_index('Id')
print('Lepto actuals per inspection 3 years prior and including year of inspection:')
print(leptoInsActualsExample)

Lepto actuals per inspection 3 years prior and including year of inspection:
    Year Lepto_Y_N      1      2      3      4   5  6    7     8  ...     39  \
Id                                                                ...          
1   2017       Yes  151.5  194.0  258.0  132.5  12  0  0.0  29.5  ...  142.0   
2   2017       Yes  219.5  181.4  323.2   79.0   0  0  0.0  20.0  ...   66.2   
3   2017       Yes  301.5  209.7  235.4   81.5   3  0  2.0  17.0  ...  105.8   
4   2017       Yes  249.0  226.2  168.3   86.5   0  0  2.7  14.5  ...  105.8   
5   2017       Yes  175.5  111.3  229.0   48.9   0  0  0.0  10.0  ...   81.0   

       40    41  42   43    44    45     46    47    48  
Id                                                       
1    54.0  45.5   0  0.3  29.7   5.6  130.8  41.6  68.2  
2   135.0  60.5   0  1.4   0.0   0.0   51.1  65.2  84.4  
3   123.5  31.5   0  1.4   0.0   0.0   51.1  65.2  84.4  
4   123.5  31.5   0  1.4   0.0   0.0   51.1  65.2  84.4  
5    71.0  32.

##### 1.4.2.2.2) Sirex

In [15]:
# #Load files
# sirexAll = pd.read_csv('C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/Modeling/Sirex_R01.csv', usecols = ['Id','Year','Sirex_Presence'])
# sirexStations = pd.read_csv(exportFolder + 'sirexStations.csv')

# #Run function
# sirexInsActuals = getActualReadings(sirexAll, sirexStations, 2012)

# #Export
# print('Sirex actuals per inspection 3 years prior and including year of inspection:')
# print(sirexInsActuals)
# sirexInsActuals.to_csv(exportFolder + 'sirexInsActuals.csv')

sirexInsActualsExample = read_from_drive('https://drive.google.com/file/d/1oIA2mVWGWyLHEZs5pQIhWsmglYH39sUm/view?usp=sharing').set_index('Id')
print('Sirex actuals per inspection 3 years prior and including year of inspection:')
print(sirexInsActualsExample)

Sirex actuals per inspection 3 years prior and including year of inspection:
     Year Sirex_Presence      1      2      3     4     5      6     7    8  \
Id                                                                            
392  2012       No Sirex  189.9  157.0  153.0  37.5   9.5   59.5  20.5  2.5   
391  2012       No Sirex  189.9  157.0  153.0  37.5   9.5   59.5  20.5  2.5   
390  2012       No Sirex  189.9  157.0  153.0  37.5   9.5   59.5  20.5  2.5   
389  2012       No Sirex  295.2  147.5  101.5   6.5   0.0   48.0  10.5  0.0   
388  2012       No Sirex  314.0  329.8  272.0  59.4  29.2  121.6  59.4  0.0   

     ...    39    40   41    42   43    44    45    46  47     48  
Id   ...                                                           
392  ...  28.0  24.0  0.0  10.0  0.0   0.0   7.0  57.4  82  124.5  
391  ...  28.0  24.0  0.0  10.0  0.0   0.0   7.0  57.4  82  124.5  
390  ...  28.0  24.0  0.0  10.0  0.0   0.0   7.0  57.4  82  124.5  
389  ...  15.0   2.8  0.0   2

## Step 5) Calculate difference between actual and average readings per month

### 5.1) Difference function

In [16]:
# def getDifference(pestActuals, pestAvgs):
#     pestDiff = pestActuals.copy(deep=True)
#     for col in range(2,len(pestActuals.columns)):
#         totalMonth = col - 1

#         if totalMonth/12 > 3:
#             month = totalMonth - 36
#         elif totalMonth/12 > 2:
#             month = totalMonth - 24
#         elif totalMonth/12 > 1:
#             month = totalMonth - 12
#         else:
#             month = totalMonth
        
#         pestDiff[str(totalMonth)] = pestActuals[str(totalMonth)] - pestAvgs[str(month)] #actuals - avg
#     return pestDiff

### 5.2) Applied to pests

#### 5.2.1) Leptocybe

In [17]:
# #Load files
# leptoInsActuals = pd.read_csv(exportFolder + 'leptoInsActuals.csv').set_index('Id')
# leptoReadingPerMonth = pd.read_csv(exportFolder + 'leptoReadingPerMonthAvg.csv').set_index('Id')

# #Run function
# leptoDiff = getDifference(leptoInsActuals, leptoReadingPerMonth)

# #Export
# print(leptoDiff.head())
# leptoDiff.to_csv(exportFolder + 'leptoDiff.csv')

leptoDiffExample = read_from_drive('https://drive.google.com/file/d/1oYuDA7ImugQqeuF5Vizt-95BXNRcyzLU/view?usp=sharing').set_index('Id')
print('Lepto actual - average 3 years prior and including year of inspection:')
print(leptoDiffExample)

Lepto actual - average 3 years prior and including year of inspection:
    Year Lepto_Y_N          1          2           3          4          5  \
Id                                                                           
1   2017       Yes -33.291304  35.892647  102.759702  62.175000 -20.157812   
2   2017       Yes -62.415942 -71.826087  146.494203 -15.202899 -31.244615   
3   2017       Yes  48.133333 -43.927536   74.576812   1.410448 -25.629508   
4   2017       Yes  16.333333  13.414493   27.797101  14.873134 -27.481356   
5   2017       Yes -28.066667 -93.675000  108.504348 -14.214493 -27.820755   

            6          7         8  ...          39         40         41  \
Id                                  ...                                     
1  -21.518966 -23.684746 -7.103030  ...  -13.240299 -16.325000  13.342187   
2  -18.326984 -20.763077  0.115625  ... -110.505797  40.797101  29.255385   
3  -17.891071 -17.112281  0.980952  ...  -55.023188  43.410448   2.870492 

#### 5.2.2) Sirex

In [18]:
# #Load files
# sirexInsActuals = pd.read_csv(exportFolder + 'sirexInsActuals.csv').set_index('Id')
# sirexReadingPerMonth = pd.read_csv(exportFolder + 'sirexReadingPerMonthAvg.csv').set_index('Id')

# #Run function
# sirexDiff = getDifference(sirexInsActuals, sirexReadingPerMonth)

# #Export
# print(sirexDiff.head())
# sirexDiff.to_csv(exportFolder + 'sirexDiff.csv')

sirexDiffExample = read_from_drive('https://drive.google.com/file/d/1qeDGrjLSWOAOOWIt6CxlRGkGspYPy3-y/view?usp=sharing').set_index('Id')
print('Sirex actual - average 3 years prior and including year of inspection:')
print(sirexDiffExample)

Sirex actual - average 3 years prior and including year of inspection:
     Year Sirex_Presence           1           2           3          4  \
Id                                                                        
392  2012       No Sirex  -64.017647  -80.117647   15.063768 -33.981538   
391  2012       No Sirex  -64.017647  -95.433824    9.521739 -35.732308   
390  2012       No Sirex  -67.923188 -108.194203   -5.786957 -37.730882   
389  2012       No Sirex  166.217391   25.791304   23.108696 -29.992424   
388  2012       No Sirex   71.920588   75.759420  122.525373  -9.397015   

             5          6          7          8  ...          39         40  \
Id                                               ...                          
392 -26.262712  32.694231  -3.389091 -21.108197  ... -109.936232 -47.481538   
391 -26.744068  31.380769  -3.554545 -21.108197  ... -115.478261 -49.232308   
390 -25.673437  33.640984  -3.324138 -20.401563  ... -130.786957 -51.230882   
389 -20.

## Step 6) Format data for modelling

### 6.1) Get format for variable of interest

#### 6.1.1) Leptocybe

In [19]:
# #Load the differences
# leptoDiff = pd.read_csv(exportFolder + 'leptoDiff.csv')
# leptoDiffTrans = leptoDiff.melt(id_vars=["Id", "Year", "Lepto_Y_N"], var_name="Timeframe", value_name=variableOfNote + 'diff')
# leptoDiffTrans = leptoDiffTrans.dropna(subset=[variableOfNote + 'diff'])
# #Load the actuals
# leptoInsActuals = pd.read_csv(exportFolder + 'leptoInsActuals.csv')
# leptoInsActuals = leptoInsActuals.drop(columns = ['Year','Lepto_Y_N'])
# leptoInsActualsTrans = leptoInsActuals.melt(id_vars=["Id"], var_name="Timeframe", value_name=variableOfNote)
# leptoInsActualsTrans = leptoInsActualsTrans.dropna(subset=[variableOfNote])
# #Join differences and actuals
# leptoDiffActTrans = leptoDiffTrans.set_index(['Id','Timeframe']).join(leptoInsActualsTrans.set_index(['Id','Timeframe'])).reset_index()

# #Load averages
# leptoReadingPerMonthAvg = pd.read_csv(exportFolder + 'leptoReadingPerMonthAvg.csv')
# leptoReadingPerMonthAvg = leptoReadingPerMonthAvg.drop(columns = ['x_m','y_m'])
# leptoReadingPerMonthAvgTrans = leptoReadingPerMonthAvg.melt(id_vars=["Id"], var_name="Timeframe", value_name=variableOfNote + 'avg')
# leptoReadingPerMonthAvgTrans = leptoReadingPerMonthAvgTrans.dropna(subset=[variableOfNote + 'avg'])
# leptoReadingPerMonthAvgTrans['Timeframe'] = pd.to_numeric(leptoReadingPerMonthAvgTrans['Timeframe'])

# for i in range(1,4):
#     leptoReadingPerMonthAvgTransP1 = leptoReadingPerMonthAvgTrans.copy()
#     leptoReadingPerMonthAvgTransP1['Timeframe'] = leptoReadingPerMonthAvgTransP1['Timeframe'] + 12*i
#     leptoReadingPerMonthAvgTrans = leptoReadingPerMonthAvgTrans.append(leptoReadingPerMonthAvgTransP1)
# leptoReadingPerMonthAvgTrans['Timeframe'] = leptoReadingPerMonthAvgTrans['Timeframe'].astype(str)

# #Merge difference, actuals and averages
# # leptoDiffActTrans = leptoDiffActTrans.set_index(['Id','Timeframe']).join(leptoReadingPerMonthAvgTrans.set_index(['Id','Timeframe']))
# leptoDiffActTrans = pd.merge(leptoDiffActTrans,leptoReadingPerMonthAvgTrans, on =['Id','Timeframe'])
# leptoDiffActTrans = leptoDiffActTrans.drop_duplicates(subset = ['Id','Timeframe'])
# leptoDiffActTrans = leptoDiffActTrans.set_index(['Id','Timeframe'])

# #Export
# leptoDiffActTrans.to_csv(exportFolder + 'leptoDiffActTrans.csv')
# print('Readings per month per Lepto inspection (expanded):')
# print(leptoDiffActTrans.head())

leptoDiffActTransExample = read_from_drive('https://drive.google.com/file/d/1z5TC3Qx_OkRrfVf861gNqZbYgf0KtzZ7/view?usp=sharing').set_index('Id')
print('Readings per month per Lepto inspection (expanded):')
print(leptoDiffActTransExample)

Readings per month per Lepto inspection (expanded):
    Timeframe  Year Lepto_Y_N  rainfalldiff  rainfall  rainfallavg
Id                                                                
1           1  2017       Yes    -33.291304     151.5   184.791304
2           1  2017       Yes    -62.415942     219.5   281.915942
3           1  2017       Yes     48.133333     301.5   253.366667
4           1  2017       Yes     16.333333     249.0   232.666667
5           1  2017       Yes    -28.066667     175.5   203.566667
6           1  2017       Yes    -24.876812     106.0   130.876812


#### 6.1.2) Sirex

In [20]:
# #Load the differences
# sirexDiff = pd.read_csv(exportFolder + 'sirexDiff.csv')
# sirexDiffTrans = sirexDiff.melt(id_vars=["Id", "Year", "Sirex_Presence"], var_name="Timeframe", value_name=variableOfNote + 'diff')
# sirexDiffTrans = sirexDiffTrans.dropna(subset=[variableOfNote + 'diff'])
# #Load the actuals
# sirexInsActuals = pd.read_csv(exportFolder + 'sirexInsActuals.csv')
# sirexInsActuals = sirexInsActuals.drop(columns = ['Year','Sirex_Presence'])
# sirexInsActualsTrans = sirexInsActuals.melt(id_vars=["Id"], var_name="Timeframe", value_name=variableOfNote)
# sirexInsActualsTrans = sirexInsActualsTrans.dropna(subset=[variableOfNote])
# #Join differences and actuals
# sirexDiffActTrans = sirexDiffTrans.set_index(['Id','Timeframe']).join(sirexInsActualsTrans.set_index(['Id','Timeframe'])).reset_index()

# #Load averages
# sirexReadingPerMonthAvg = pd.read_csv(exportFolder + 'sirexReadingPerMonthAvg.csv')
# sirexReadingPerMonthAvg = sirexReadingPerMonthAvg.drop(columns = ['x_m','y_m'])
# sirexReadingPerMonthAvgTrans = sirexReadingPerMonthAvg.melt(id_vars=["Id"], var_name="Timeframe", value_name=variableOfNote + 'avg')
# sirexReadingPerMonthAvgTrans = sirexReadingPerMonthAvgTrans.dropna(subset=[variableOfNote + 'avg'])
# sirexReadingPerMonthAvgTrans['Timeframe'] = pd.to_numeric(sirexReadingPerMonthAvgTrans['Timeframe'])

# for i in range(1,4):
#     sirexReadingPerMonthAvgTransP1 = sirexReadingPerMonthAvgTrans.copy()
#     sirexReadingPerMonthAvgTransP1['Timeframe'] = sirexReadingPerMonthAvgTransP1['Timeframe'] + 12*i
#     sirexReadingPerMonthAvgTrans = sirexReadingPerMonthAvgTrans.append(sirexReadingPerMonthAvgTransP1)
# sirexReadingPerMonthAvgTrans['Timeframe'] = sirexReadingPerMonthAvgTrans['Timeframe'].astype(str)

# #Merge difference, actuals and averages
# sirexDiffActTrans = pd.merge(sirexDiffActTrans,sirexReadingPerMonthAvgTrans, on =['Id','Timeframe'])
# sirexDiffActTrans = sirexDiffActTrans.drop_duplicates(subset = ['Id','Timeframe'])
# sirexDiffActTrans = sirexDiffActTrans.set_index(['Id','Timeframe'])

# #Export
# sirexDiffActTrans.to_csv(exportFolder + 'sirexDiffActTrans.csv')
# print('Readings per month per Sirex inspection (expanded):')
# print(sirexDiffActTrans.head())


sirexDiffActTransExample = read_from_drive('https://drive.google.com/file/d/1dprWesS_8NfmFQDF_ehLVpQ3sG86Lw2e/view?usp=sharing').set_index('Id')
print('Readings per month per Sirex inspection (expanded):')
print(sirexDiffActTransExample)

Readings per month per Sirex inspection (expanded):
     Timeframe  Year Sirex_Presence  rainfalldiff  rainfall  rainfallavg
Id                                                                      
392          1  2012       No Sirex    -64.017647     189.9   253.917647
391          1  2012       No Sirex    -64.017647     189.9   253.917647
390          1  2012       No Sirex    -67.923188     189.9   257.823188
389          1  2012       No Sirex    166.217391     295.2   128.982609
388          1  2012       No Sirex     71.920588     314.0   242.079412
387          1  2012       No Sirex    100.532308     366.5   265.967692
386          1  2012       No Sirex     51.104348     314.0   262.895652
384          1  2012       No Sirex    -42.729851     209.0   251.729851
385          1  2012       No Sirex    -23.948485     213.8   237.748485


### 6.2) Merge files

Run Step1 1 to 6.1 first, for rainfall, max and min temp, and temp diff

#### 6.2.1) Function to merge

In [21]:
# def mergeData(pestDataOg, filename):
#     baseFolder = 'C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/Modeling/'
#     getVariable = ['rainfall','temperature_max','temperature_min','temperature_diff']   
#     pestDataOg = pestDataOg[pestDataOg['Year']<2019] #Limit due to available weather data 

#     pestData = pestDataOg.copy(deep=True)
#     pestData['Timeframe'] = 1

#     for i in range(2,49):
#         pestDataNew = pestDataOg.copy(deep=True)
#         pestDataNew['Timeframe'] = i
#         pestData = pestData.append(pestDataNew)
#     pestData = pestData.set_index(['Id','Timeframe'])

#     for v in getVariable:
#         pestDiffTrans = pd.read_csv(baseFolder + v + '/' + filename, usecols = ['Id','Timeframe', v +'diff', v, v +'avg'])  
#         pestData = pestData.join(pestDiffTrans.set_index(['Id','Timeframe']))

#     pestData = pestData.reset_index()
#     return pestData

#### 6.2.2) Applied to pests

In [22]:
# exportFolder = 'C:/Users/gf2704928/Documents/_MIT (Desktop)/__DataFor808/Modeling/'
# leptoDataOg = pd.read_csv (r"C:/Users/gf2704928\Documents/_MIT (Desktop)/__DataFor808/Modeling/Leptocybe_R01.csv", usecols = ['Id','Year','Lepto_Y_N','% Infestation'])
# leptoData = mergeData(leptoDataOg,'leptoDiffActTrans.csv')
# leptoData.to_csv(exportFolder + 'leptoDataCombined.csv')

leptoData = read_from_drive('https://drive.google.com/file/d/1GvzuS_mQkL7bBGu8jRn1IOehLcGX__zf/view?usp=sharing').set_index(['Id','Timeframe'])
print('Lepto data:')
print(leptoData.head())

Lepto data:
              Unnamed: 0  Year Lepto_Y_N  % Infestation  rainfalldiff  \
Id Timeframe                                                            
1  1                   0  2017       Yes           72.0    -33.291304   
2  1                   1  2017       Yes           92.0    -62.415942   
3  1                   2  2017       Yes           98.0     48.133333   
4  1                   3  2017       Yes          100.0     16.333333   
5  1                   4  2017       Yes          100.0    -28.066667   

              rainfall  rainfallavg  temperature_maxdiff  temperature_max  \
Id Timeframe                                                                
1  1             151.5   184.791304             2.226002        29.335484   
2  1             219.5   281.915942             0.652248        28.432258   
3  1             301.5   253.366667             0.563832        28.432258   
4  1             249.0   232.666667             0.563832        28.432258   
5  1          

In [23]:
# sirexDataOg = pd.read_csv (r"C:/Users/gf2704928\Documents/_MIT (Desktop)/__DataFor808/Modeling/Sirex_R01.csv", usecols = ['Id','Year','Sirex_Presence'])
# sirexData = mergeData(sirexDataOg,'sirexDiffActTrans.csv')
# sirexData.to_csv(exportFolder + 'sirexDataCombined.csv')

sirexData = read_from_drive('https://drive.google.com/file/d/1nM4cA8AReB7OJaZUfl04tdSioDnyuKpZ/view?usp=sharing').set_index(['Id','Timeframe'])
print('Sirex data:')
print(sirexData.head())

Sirex data:
               Unnamed: 0  Year Sirex_Presence  rainfalldiff  rainfall  \
Id  Timeframe                                                            
392 1                   0  2012       No Sirex    -64.017647     189.9   
391 1                   1  2012       No Sirex    -64.017647     189.9   
390 1                   2  2012       No Sirex    -67.923188     189.9   
389 1                   3  2012       No Sirex    166.217391     295.2   
388 1                   4  2012       No Sirex     71.920588     314.0   

               rainfallavg  temperature_maxdiff  temperature_max  \
Id  Timeframe                                                      
392 1           253.917647             2.981476        30.054839   
391 1           253.917647             2.981476        30.054839   
390 1           257.823188             2.981476        30.054839   
389 1           128.982609             2.851955        30.054839   
388 1           242.079412             1.874780        30.054